In [155]:
import pandas as pd
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.utils import resample
from sklearn.metrics import precision_recall_fscore_support, precision_recall_curve, precision_recall_curve, classification_report
from sklearn.linear_model import LogisticRegression

In [3]:
df = pd.read_csv('h1b_application.csv', index_col=0)

In [4]:
df.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,EMPLOYER_NAME,EMPLOYER_NUM_EMPLOYEES,WORKSITE_CITY,WORKSITE_STATE,WAGE_OFFER_FROM,JOB_TITLE,COUNTRY_OF_CITIZENSHIP,FOREIGN_WORKER_EDUCATION,FOREIGN_WORKER_INFO_MAJOR,RECEIVED_YEAR,RECEIVED_MONTH,DECISION_YEAR,DECISION_MONTH,WAIT_TIME_IN_DAYS
0,A-18038-41047,Denied,2018-03-26,2018-10-01,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,95618.0,Business Intellingence Analyst,INDIA,Bachelor's,BIOINFORMATICS,2018,3,2018,10,189
1,A-18039-41598,Denied,2018-03-27,2018-10-01,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,99840.0,"Software Developers, System Software",INDIA,Master's,COMPUTER SCIENCE,2018,3,2018,10,188
2,A-18051-45341,Denied,2018-03-26,2018-10-01,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,99840.0,"Software Developers, System Software",INDIA,Bachelor's,CIVIL ENGINEERING,2018,3,2018,10,189
3,A-18068-51106,Denied,2018-03-22,2018-10-01,LAUNCH IT CORP,54,Urbandale,IOWA,103958.0,Computer Programmer,INDIA,Bachelor's,MECHANICAL ENGINEERING,2018,3,2018,10,193
4,A-18205-00752,Denied,2018-08-05,2018-10-01,CONSUMERS ENERGY,7175,Ludington,MICHIGAN,110040.0,Project Manager,CANADA,Bachelor's,MECHANICAL ENGINEERING,2018,8,2018,10,57


In [5]:
df.EMPLOYER_NUM_EMPLOYEES = df.EMPLOYER_NUM_EMPLOYEES.astype(int)

In [6]:
df['RECEIVED_DATE'] = pd.to_datetime(df['RECEIVED_DATE'], format = '%Y-%m-%d')
df['DECISION_DATE'] = pd.to_datetime(df['DECISION_DATE'], format = '%Y-%m-%d')

In [7]:
df['RECEIVED_YEAR'] = df['RECEIVED_DATE'].dt.year
df['RECEIVED_MONTH'] = df['RECEIVED_DATE'].dt.month
df['DECISION_YEAR'] = df['DECISION_DATE'].dt.year
df['DECISION_MONTH'] = df['DECISION_DATE'].dt.month

In [8]:
df['WAIT_TIME_IN_DAYS'] = (df.DECISION_DATE - df.RECEIVED_DATE).dt.days

In [9]:
df.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,EMPLOYER_NAME,EMPLOYER_NUM_EMPLOYEES,WORKSITE_CITY,WORKSITE_STATE,WAGE_OFFER_FROM,JOB_TITLE,COUNTRY_OF_CITIZENSHIP,FOREIGN_WORKER_EDUCATION,FOREIGN_WORKER_INFO_MAJOR,RECEIVED_YEAR,RECEIVED_MONTH,DECISION_YEAR,DECISION_MONTH,WAIT_TIME_IN_DAYS
0,A-18038-41047,Denied,2018-03-26,2018-10-01,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,95618.0,Business Intellingence Analyst,INDIA,Bachelor's,BIOINFORMATICS,2018,3,2018,10,189
1,A-18039-41598,Denied,2018-03-27,2018-10-01,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,99840.0,"Software Developers, System Software",INDIA,Master's,COMPUTER SCIENCE,2018,3,2018,10,188
2,A-18051-45341,Denied,2018-03-26,2018-10-01,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,99840.0,"Software Developers, System Software",INDIA,Bachelor's,CIVIL ENGINEERING,2018,3,2018,10,189
3,A-18068-51106,Denied,2018-03-22,2018-10-01,LAUNCH IT CORP,54,Urbandale,IOWA,103958.0,Computer Programmer,INDIA,Bachelor's,MECHANICAL ENGINEERING,2018,3,2018,10,193
4,A-18205-00752,Denied,2018-08-05,2018-10-01,CONSUMERS ENERGY,7175,Ludington,MICHIGAN,110040.0,Project Manager,CANADA,Bachelor's,MECHANICAL ENGINEERING,2018,8,2018,10,57


In [10]:
df.to_csv('h1b_application.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123798 entries, 0 to 123862
Data columns (total 18 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   CASE_NUMBER                123798 non-null  object        
 1   CASE_STATUS                123798 non-null  object        
 2   RECEIVED_DATE              123798 non-null  datetime64[ns]
 3   DECISION_DATE              123798 non-null  datetime64[ns]
 4   EMPLOYER_NAME              123798 non-null  object        
 5   EMPLOYER_NUM_EMPLOYEES     123798 non-null  int64         
 6   WORKSITE_CITY              123798 non-null  object        
 7   WORKSITE_STATE             123798 non-null  object        
 8   WAGE_OFFER_FROM            123798 non-null  float64       
 9   JOB_TITLE                  123798 non-null  object        
 10  COUNTRY_OF_CITIZENSHIP     123798 non-null  object        
 11  FOREIGN_WORKER_EDUCATION   123798 non-null  object  

In [11]:
df1 = df.drop(['RECEIVED_DATE', 'DECISION_DATE'], axis=1)

In [16]:
df2 = pd.get_dummies(df1, columns = ['CASE_STATUS', 'EMPLOYER_NAME', 'WORKSITE_CITY', 'WORKSITE_STATE','JOB_TITLE',
       'COUNTRY_OF_CITIZENSHIP', 'FOREIGN_WORKER_EDUCATION','FOREIGN_WORKER_INFO_MAJOR'], drop_first=True)

In [18]:
df2.drop('CASE_NUMBER', axis=1, inplace=True)

In [19]:
df2.head()

,EMPLOYER_NUM_EMPLOYEES,WAGE_OFFER_FROM,RECEIVED_YEAR,RECEIVED_MONTH,DECISION_YEAR,DECISION_MONTH,WAIT_TIME_IN_DAYS,CASE_STATUS_Denied,EMPLOYER_NAME_1010DATA SERVICES LLC,EMPLOYER_NAME_10Bits LLC,...,FOREIGN_WORKER_INFO_MAJOR_WON BUDDHISM,FOREIGN_WORKER_INFO_MAJOR_WOOD SCIENCE,FOREIGN_WORKER_INFO_MAJOR_WOOD SCIENCE TECHNOLOGY,FOREIGN_WORKER_INFO_MAJOR_WORKFORCE EDUCATION AND DEVELOPMENT,FOREIGN_WORKER_INFO_MAJOR_WORLD LANGUAGE EDUCATION,FOREIGN_WORKER_INFO_MAJOR_WORLD LANGUAGES,FOREIGN_WORKER_INFO_MAJOR_YESHIVA UNIVERSITY,FOREIGN_WORKER_INFO_MAJOR_ZOOLOGY,"FOREIGN_WORKER_INFO_MAJOR_ZOOLOGY, CHEMISTRY, MICROBIOLOGY",FOREIGN_WORKER_INFO_MAJOR_ZOOTECHNICS
0,59,95618.0,2018,3,2018,10,189,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,59,99840.0,2018,3,2018,10,188,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,59,99840.0,2018,3,2018,10,189,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,54,103958.0,2018,3,2018,10,193,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,7175,110040.0,2018,8,2018,10,57,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
df1.head()

,CASE_NUMBER,CASE_STATUS,EMPLOYER_NAME,EMPLOYER_NUM_EMPLOYEES,WORKSITE_CITY,WORKSITE_STATE,WAGE_OFFER_FROM,JOB_TITLE,COUNTRY_OF_CITIZENSHIP,FOREIGN_WORKER_EDUCATION,FOREIGN_WORKER_INFO_MAJOR,RECEIVED_YEAR,RECEIVED_MONTH,DECISION_YEAR,DECISION_MONTH,WAIT_TIME_IN_DAYS
0,A-18038-41047,Denied,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,95618.0,Business Intellingence Analyst,INDIA,Bachelor's,BIOINFORMATICS,2018,3,2018,10,189
1,A-18039-41598,Denied,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,99840.0,"Software Developers, System Software",INDIA,Master's,COMPUTER SCIENCE,2018,3,2018,10,188
2,A-18051-45341,Denied,NAZTEC INTERNATIONAL GROUP LLC,59,West Palm Beach,FLORIDA,99840.0,"Software Developers, System Software",INDIA,Bachelor's,CIVIL ENGINEERING,2018,3,2018,10,189
3,A-18068-51106,Denied,LAUNCH IT CORP,54,Urbandale,IOWA,103958.0,Computer Programmer,INDIA,Bachelor's,MECHANICAL ENGINEERING,2018,3,2018,10,193
4,A-18205-00752,Denied,CONSUMERS ENERGY,7175,Ludington,MICHIGAN,110040.0,Project Manager,CANADA,Bachelor's,MECHANICAL ENGINEERING,2018,8,2018,10,57


In [117]:
df3 = df1.drop(['WORKSITE_CITY','CASE_NUMBER'], axis=1)

In [118]:
drop_job = df3.JOB_TITLE.value_counts()[df3.JOB_TITLE.value_counts()<10].index

In [119]:
drop_major = df3.FOREIGN_WORKER_INFO_MAJOR.value_counts()[df3.FOREIGN_WORKER_INFO_MAJOR.value_counts()<10].index

In [123]:
drop_company = df3.EMPLOYER_NAME.value_counts()[df3.EMPLOYER_NAME.value_counts()<10].index

In [124]:
df3 = df3[~df3.JOB_TITLE.isin(drop_job)]
df3 = df3[~df3.FOREIGN_WORKER_INFO_MAJOR.isin(drop_major)]
df3 = df3[~df3.EMPLOYER_NAME.isin(drop_company)]

In [127]:
df3.drop(['EMPLOYER_NUM_EMPLOYEES','RECEIVED_YEAR','RECEIVED_MONTH','DECISION_YEAR','DECISION_MONTH','WAIT_TIME_IN_DAYS'], axis=1, inplace=True)

In [240]:
df3.to_csv('data_ml.csv')

In [167]:
df3.columns

Index(['CASE_STATUS', 'EMPLOYER_NAME', 'WORKSITE_STATE', 'WAGE_OFFER_FROM',
       'JOB_TITLE', 'COUNTRY_OF_CITIZENSHIP', 'FOREIGN_WORKER_EDUCATION',
       'FOREIGN_WORKER_INFO_MAJOR'],
      dtype='object')

In [130]:
df4 = pd.get_dummies(df3, columns = ['JOB_TITLE','EMPLOYER_NAME','WORKSITE_STATE',
       'COUNTRY_OF_CITIZENSHIP', 'FOREIGN_WORKER_EDUCATION','FOREIGN_WORKER_INFO_MAJOR'], drop_first = True)

In [131]:
df4= pd.get_dummies(df4, columns = ['CASE_STATUS'])

In [132]:
df4.drop('CASE_STATUS_Denied', axis = 1, inplace=True)

In [133]:
df4.value_counts('CASE_STATUS_Certified')

CASE_STATUS_Certified
1    52365
0     1001
dtype: int64

Imblance data

In [93]:
y = df4.CASE_STATUS_Certified
X = df4.drop('CASE_STATUS_Certified', axis = 1)

In [94]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=66)

In [95]:
rf = RandomForestClassifier(max_depth=10)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_pred_prob = rf.predict_proba(X_test)

In [173]:
pd.Series(X_test.columns).to_csv('Model column.csv')

In [96]:
y_pred_prob

array([[0.01840537, 0.98159463],
       [0.02184075, 0.97815925],
       [0.02301249, 0.97698751],
       ...,
       [0.01781937, 0.98218063],
       [0.02182797, 0.97817203],
       [0.01982019, 0.98017981]])

In [97]:
accuracy_score(y_test, y_pred)

0.9716950788034738

In [98]:
confusion_matrix(y_test, y_pred)

array([[    6,   616],
       [    0, 21141]])

In [99]:
mean_squared_error(y_test, y_pred)

0.028304921196526216

In [70]:
df3.value_counts('CASE_STATUS')

CASE_STATUS
Certified    70408
Denied        2133
dtype: int64

In [100]:
precision_recall_curve(y_test, y_pred)

(array([0.97168727, 1.        ]), array([1., 0.]), array([1], dtype=uint8))

In [74]:
precision_recall_fscore_support(y_test, y_pred)

(array([0.97150866, 1.        ]),
 array([1.        , 0.00321543]),
 array([0.98554846, 0.00641026]),
 array([21141,   622]))

In [134]:
df4

,WAGE_OFFER_FROM,JOB_TITLE_Account Manager,JOB_TITLE_Account Strategist,JOB_TITLE_Accountant,JOB_TITLE_Accounting Manager,JOB_TITLE_Actuarial Analyst,JOB_TITLE_Actuary,JOB_TITLE_Advanced Software Developer,JOB_TITLE_Advisory Manager,"JOB_TITLE_Advisory Manager, Application Technology",...,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS AND SOFTWARE ENGINEERING,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS ENGINEERING,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS ENGINEERING TECHNOLOGY,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS SYSTEMS MANAGEMENT,FOREIGN_WORKER_INFO_MAJOR_TEXTILE TECHNOLOGY,FOREIGN_WORKER_INFO_MAJOR_URBAN DESIGN,FOREIGN_WORKER_INFO_MAJOR_URBAN PLANNING,FOREIGN_WORKER_INFO_MAJOR_VETERINARY MEDICINE,FOREIGN_WORKER_INFO_MAJOR_WEB DESIGN NEW MEDIA,CASE_STATUS_Certified
10,133000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,118000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,98010.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,66000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,99382.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123840,171000.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
123850,111966.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
123855,121306.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
123859,93558.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [136]:
certified = df4[df4['CASE_STATUS_Certified'] == 1]
denied = df4[df4['CASE_STATUS_Certified'] == 0]

In [137]:
certified_downsample = resample(certified,
             replace=True,
             n_samples=len(denied),
             random_state=42)
df5 = pd.concat([certified_downsample, denied])

In [138]:
y = df5['CASE_STATUS_Certified']
X = df5.drop('CASE_STATUS_Certified', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=66)

In [139]:
rf = RandomForestClassifier(max_depth=30)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
y_pred_prob = rf.predict_proba(X_test)

In [140]:
mean_squared_error(y_test, y_pred)

0.22129783693843594

In [141]:
accuracy_score(y_test, y_pred)

0.778702163061564

In [151]:
report = classification_report(y_test, y_pred)

In [153]:
print(report)

              precision    recall  f1-score   support

           0       0.83      0.66      0.73       278
           1       0.75      0.88      0.81       323

    accuracy                           0.78       601
   macro avg       0.79      0.77      0.77       601
weighted avg       0.79      0.78      0.78       601



In [164]:
log = LogisticRegression().fit(X_train, y_train)
y_log_pred = log.predict(X_test)

In [166]:
mean_squared_error(y_test, y_log_pred)

0.46256239600665555

In [154]:
import joblib

joblib.dump(rf, "rf.pkl")

['rf.pkl']

In [197]:
import numpy as np

,0,1,2,3,4,5,6,7,8,9,...,2602,2603,2604,2605,2606,2607,2608,2609,2610,2611
0,7,7,5,5,3,6,2,8,6,3,...,8,6,6,0,7,2,4,8,8,5


In [220]:
data_columns = X.columns
zero = [0]*len(data_columns)

pd.DataFrame(np.zeros(shape=(1,len(data_columns))), columns=data_columns)

,WAGE_OFFER_FROM,JOB_TITLE_Account Manager,JOB_TITLE_Account Strategist,JOB_TITLE_Accountant,JOB_TITLE_Accounting Manager,JOB_TITLE_Actuarial Analyst,JOB_TITLE_Actuary,JOB_TITLE_Advanced Software Developer,JOB_TITLE_Advisory Manager,"JOB_TITLE_Advisory Manager, Application Technology",...,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS AND NETWORK MANAGEMENT,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS AND SOFTWARE ENGINEERING,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS ENGINEERING,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS ENGINEERING TECHNOLOGY,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS SYSTEMS MANAGEMENT,FOREIGN_WORKER_INFO_MAJOR_TEXTILE TECHNOLOGY,FOREIGN_WORKER_INFO_MAJOR_URBAN DESIGN,FOREIGN_WORKER_INFO_MAJOR_URBAN PLANNING,FOREIGN_WORKER_INFO_MAJOR_VETERINARY MEDICINE,FOREIGN_WORKER_INFO_MAJOR_WEB DESIGN NEW MEDIA
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [228]:
test = pd.DataFrame(np.zeros(shape=(1,len(data_columns))), columns=data_columns)

In [232]:
employer_selection = '1 Stop Electronics Center Inc'
state_selection='ALABAMA'
job_selection='Software Engineer'
country_selection='TAIWAN'
education_selection="Master's"
major_selection='BIOINFORMATICS'
wage = 1000

In [233]:
test['EMPLOYER_NAME_'+employer_selection] = 1
test['WORKSITE_STATE_'+state_selection] = 1
test['JOB_TITLE_'+job_selection] = 1
test['COUNTRY_OF_CITIZENSHIP_'+country_selection] = 1
test['FOREIGN_WORKER_EDUCATION_'+education_selection] = 1
test['FOREIGN_WORKER_INFO_MAJOR_'+major_selection] = 1
test['WAGE_OFFER_FROM'] = wage

In [234]:
test

,WAGE_OFFER_FROM,JOB_TITLE_Account Manager,JOB_TITLE_Account Strategist,JOB_TITLE_Accountant,JOB_TITLE_Accounting Manager,JOB_TITLE_Actuarial Analyst,JOB_TITLE_Actuary,JOB_TITLE_Advanced Software Developer,JOB_TITLE_Advisory Manager,"JOB_TITLE_Advisory Manager, Application Technology",...,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS ENGINEERING TECHNOLOGY,FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS SYSTEMS MANAGEMENT,FOREIGN_WORKER_INFO_MAJOR_TEXTILE TECHNOLOGY,FOREIGN_WORKER_INFO_MAJOR_URBAN DESIGN,FOREIGN_WORKER_INFO_MAJOR_URBAN PLANNING,FOREIGN_WORKER_INFO_MAJOR_VETERINARY MEDICINE,FOREIGN_WORKER_INFO_MAJOR_WEB DESIGN NEW MEDIA,EMPLOYER_NAME_1 Stop Electronics Center Inc,WORKSITE_STATE_ALABAMA,FOREIGN_WORKER_EDUCATION_Associate's
0,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,1


In [235]:
df5.to_csv('downsampled_application.csv')

In [236]:
df5.columns

Index(['WAGE_OFFER_FROM', 'JOB_TITLE_Account Manager',
       'JOB_TITLE_Account Strategist', 'JOB_TITLE_Accountant',
       'JOB_TITLE_Accounting Manager', 'JOB_TITLE_Actuarial Analyst',
       'JOB_TITLE_Actuary', 'JOB_TITLE_Advanced Software Developer',
       'JOB_TITLE_Advisory Manager',
       'JOB_TITLE_Advisory Manager, Application Technology',
       ...
       'FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS AND SOFTWARE ENGINEERING',
       'FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS ENGINEERING',
       'FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS ENGINEERING TECHNOLOGY',
       'FOREIGN_WORKER_INFO_MAJOR_TELECOMMUNICATIONS SYSTEMS MANAGEMENT',
       'FOREIGN_WORKER_INFO_MAJOR_TEXTILE TECHNOLOGY',
       'FOREIGN_WORKER_INFO_MAJOR_URBAN DESIGN',
       'FOREIGN_WORKER_INFO_MAJOR_URBAN PLANNING',
       'FOREIGN_WORKER_INFO_MAJOR_VETERINARY MEDICINE',
       'FOREIGN_WORKER_INFO_MAJOR_WEB DESIGN  NEW MEDIA',
       'CASE_STATUS_Certified'],
      dtype='object', length=2613